In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import glob
import pickle
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
#**IMPORTING DATA**
files=glob.glob('../../data/price_index_data/mod/*')

#getting the prices history from each index
data=[]
for file in files:
    data.append(pd.read_csv(file))

#converting the 'date' columns to datetime type
for i in data:
    i['date']=pd.to_datetime(i['date'])

#combining the data together
df=data[0]
for i in data[1:]:
    df=df.merge(i,on='date')

#removing the date column and setting it as the index
df.index=df['date']
df.drop(columns=['date'],inplace=True)

#removing unecessary columns
cols=[i for i in list(df.columns) if 'Unnamed' not in i]
df=df[cols]
df=df.dropna()

In [3]:
cols=[i for i in df.columns if 'close' in i]
df=df[cols]

In [4]:
with open('../../data/train/inear_log(price_data).pkl', 'rb') as f:
    data=pickle.load(f)

In [5]:
data['bit_close']

,nas_close,eth_close,dow_close,ada_close,amd_close,snp_close,bnb_close,nvda_close,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10
0,6516.72,300.062000,22557.60,0.025932,12.710000,2526.3,1.964670,44.750000,4307.555000,4186.783740,4325.236250,4370.168750,4789.301250,4741.708750,4858.604990,5728.547500,5666.978750,5643.553750
1,6531.71,292.847181,22641.67,0.020816,13.420000,2532.8,1.820700,44.842499,4186.783740,4325.236250,4370.168750,4789.301250,4741.708750,4858.604990,5728.547500,5666.978750,5643.553750,5495.038750
2,6534.63,289.300806,22661.64,0.021931,13.310000,2536.2,1.601480,45.217499,4325.236250,4370.168750,4789.301250,4741.708750,4858.604990,5728.547500,5666.978750,5643.553750,5495.038750,5627.086250
3,6585.36,295.510772,22775.39,0.021489,13.340000,2549.9,1.678490,45.192501,4370.168750,4789.301250,4741.708750,4858.604990,5728.547500,5666.978750,5643.553750,5495.038750,5627.086250,5686.526250
4,6590.18,309.599509,22773.67,0.018539,13.230000,2545.1,1.579910,45.325001,4789.301250,4741.708750,4858.604990,5728.547500,5666.978750,5643.553750,5495.038750,5627.086250,5686.526250,6098.115000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,15041.86,3217.343113,35405.50,2.738132,108.300003,4483.2,502.786255,222.130005,47329.149720,49033.325396,47663.022501,46919.511555,48487.011189,49725.532134,49709.611994,46897.601477,46769.314391,46404.364481
965,14945.81,3135.122534,35213.12,2.535528,107.269997,4483.2,479.006104,220.679993,49033.325396,47663.022501,46919.511555,48487.011189,49725.532134,49709.611994,46897.601477,46769.314391,46404.364481,44720.755550
966,15129.50,3269.132781,35455.80,2.944197,111.400002,4483.2,494.498688,226.360001,47663.022501,46919.511555,48487.011189,49725.532134,49709.611994,46897.601477,46769.314391,46404.364481,44720.755550,45097.797218
967,15265.89,3280.411209,35399.84,2.733160,111.320000,4525.2,459.490753,226.880005,46919.511555,48487.011189,49725.532134,49709.611994,46897.601477,46769.314391,46404.364481,44720.755550,45097.797218,47039.949931


In [6]:
preds=pd.DataFrame()
targets=['bit_close','eth_close','ada_close','bnb_close']
for target in targets:
    ys=[i for i in data[target].columns if 'day' in i]
    x=data[target].drop(columns=ys)
    temp=[]
    for i in ys:
        x=sm.add_constant(x)
        y=data[target][i]
        X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 1,shuffle = False)
        model= Ridge(fit_intercept=False)
        model.fit(X_train, y_train)
        temp.append(model.predict(X_test)[0])
    preds[target]=temp

In [8]:
targets=['bit_close','eth_close','ada_close','bnb_close']
tar=df[targets][-10:]
for target in targets:
    temp_y=tar[target]
    y_hat=[i for i in preds.columns if target in i]
    y_hat=preds[y_hat]
    sse=0
    for i in range(len(temp_y)):
        sse+=(temp_y.values[i]-y_hat.values[i][0])**2
    aic=sse/16
    aic=16*np.log(aic)+2
    print(target,'AIC:',aic)

bit_close AIC: 260.7661684342751
eth_close AIC: 166.58991730567044
ada_close AIC: -29.28074634383734
bnb_close AIC: 137.4092325659965


In [9]:
(260.7661684342751 + 166.58991730567044 + -29.28074634383734 + 137.4092325659965)/4


133.87114299052615